<a href="https://colab.research.google.com/github/ibtissam01/Text_Mining/blob/main/TP_TS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Partie I : Implémenter le modèle de TS T5 afin de résumer le texte.


In [2]:
!pip install transformers
from transformers import AutoTokenizer
from transformers import AutoModelWithLMHead

In [3]:
content = '''China’s Huawei overtook Samsung Electronics as the world’s biggest seller of mobile phones
in the second quarter of 2020, shipping 55.8 million devices compared to Samsung’s 53.7 million,
according to data from research firm Canalys. While Huawei’s sales fell 5 per cent from the same quarter
a year earlier, South Korea’s Samsung posted a bigger drop of 30 per cent, owing to disruption from the
coronavirus in key markets such as Brazil, the United States and Europe, Canalys said. Huawei’s overseas
shipments fell 27 per cent in Q2 from a year earlier, but the company increased its dominance of the
China market which has been faster to recover from COVID-19 and where it now sells over 70 per cent
of its phones. “Our business has demonstrated exceptional resilience in these difficult times,” a Huawei
spokesman said. “Amidst a period of unprecedented global economic slowdown and challenges, we’re
continued to grow and further our leadership position.” Nevertheless, Huawei’s position as number one
seller may prove short-lived once other markets recover given it is mainly due to economic disruption, a
senior Huawei employee with knowledge of the matter told Reuters. Apple is due to release its Q2
iPhone shipment data on Friday.'''


In [4]:
tokenizer=AutoTokenizer.from_pretrained('T5-base')
model=AutoModelWithLMHead.from_pretrained('T5-base', return_dict=True)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1509: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [6]:
inputs=tokenizer.encode("sumarize: " +content,return_tensors='pt', max_length=512,
truncation=True)

In [7]:
output = model.generate(inputs, min_length=80, max_length=100)
summary=tokenizer.decode(output[0])
print(summary)


<pad> Huawei overtook Samsung Electronics as the world’s biggest seller of mobile phones in the second quarter of 2020. shipping 55.8 million devices compared to Samsung’s 53.7 million, according to data from research firm Canalys. while Huawei’s sales fell 5 per cent from the same quarter a year earlier, south Korea’s Samsung posted a bigger drop of 30 per cent owing to disruption from the coronavirus.</s>


#Partie II : Implémenter le modèle TextRank afin de résumer le texte.

In [8]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re
import networkx as nx

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
data = pd.read_csv('tennis_articles.csv', encoding='latin-1')

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
# Prétraitement des données
def preprocess_text(text):
    # Retirer les caractères spéciaux et les chiffres
    text = re.sub('[^a-zA-Z]', ' ', text)

    # Convertir en minuscules
    text = text.lower()

    # Tokenization
    tokens = nltk.word_tokenize(text)

    # Retirer les mots vides
    stop_words = nltk.corpus.stopwords.words('english')
    tokens = [token for token in tokens if token not in stop_words]

    # Retourner le texte prétraité
    return ' '.join(tokens)

# Appliquer le prétraitement aux données
data['preprocessed_text'] = data['article_text'].apply(preprocess_text)

In [17]:
# Encodage du texte
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['preprocessed_text'])


In [19]:
# Construction du graphe
similarity_matrix = (X * X.T).toarray()
nx_graph = nx.from_numpy_array(similarity_matrix)

In [20]:
# Appliquer l'algorithme TextRank
scores = nx.pagerank(nx_graph)

In [22]:
# Trier les phrases par score
ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(data['article_text'])), reverse=True)

In [23]:
# Générer le résumé final
summary_length = 3  # Longueur du résumé en nombre de phrases

summary_sentences = [sentence for _, sentence in ranked_sentences[:summary_length]]
summary = ' '.join(summary_sentences)

# Afficher le résumé
print(summary)

I PLAYED golf last week with Todd Reid. He picked me up at 5.30am, half an hour early because he couldnt sleep. Or hadnt slept, to be specific. Not because hed been out on a bender or anything  those days were in the past. The former Wimbledon junior champion was full of hope, excited about getting his life back together after a troubled few years and a touch-and-go battle with pancreatitis. Im pleased with that, he said after grinding out an eight-over-par front nine at the not-so-royal Northbridge club in Sydney and smashing down an egg- and-bacon roll at the halfway house. To most players of his rare sporting gifts, such a modest return would be unacceptable. To Reid the 15-marker, just being up and at em was enough; a few bogeys and one well-made par  broomstick putter and all  vindication for his recent decision to renew his membership at nearby Bankstown. Exhausted after spending half his round deep in the bushes searching for my ball, as well as those of two other golf